In [2]:
import pandas as pd
import numpy as np

In [2]:
# Reunir datos en una unica version por sector
sector = ['01','02','03','04','05','06','07','08','09','11','12']
versiones = [2,2,1,1,10,2,1,1,1,1,1]
for i in range(len(sector)):
    archivo_base = 'Tagged_Tweets_' + sector[i]
    archivo_final =  'Tweets_' + sector[i] + '.csv'
    if versiones[i] == 1:
        df = pd.read_csv(archivo_base + '.csv', index_col=0)
        df.to_csv(archivo_final, index=True, header=True)
    else:
        df = pd.read_csv(archivo_base + '_' + str(1) + '.csv', index_col=0)
        df.to_csv(archivo_final, index=True, header=True)
        for j in range(1,versiones[i]):
            archivo = archivo_base + '_' + str(j+1) + '.csv'
            tweet_df = pd.read_csv(archivo, index_col=0)
            tweet_df.to_csv(archivo_final, mode='a', index=True, header=False)

In [3]:
# Eliminar columnas descartadas + formatear ciertas columnas
def FormatearDatos(nombre_archivo, sector):
    tweet_df = pd.read_csv(nombre_archivo, index_col=0)
    tweet_df.drop(["cashtags", "username"], inplace=True, axis=1)
    tweet_df.reply_to = tweet_df.reply_to.map({"{\'user_id\': None, \'username\': None}": 'No'}).fillna("Si").astype(str)
    tweet_df.place = tweet_df.place.map({np.nan: "No"}).fillna("Si").astype(str)
    tweet_df["sector"] = sector
    dia_semana = []
    dia_semana.clear()
    lemma_tag = []
    lemma_tag.clear()
    clean_tag = []
    clean_tag.clear()
    short = []
    short.clear()
    for date, lemma, clean, tweet in zip(tweet_df["date"], tweet_df["lemma_tag"], tweet_df["clean_tag"], tweet_df["clean_tweet"]):
        dateframe = pd.Timestamp(date).dayofweek
        if dateframe == 0:
            dateframe = "Lunes"
        if dateframe == 1:
            dateframe = "Martes"
        if dateframe == 2:
            dateframe = "Miércoles"
        if dateframe == 3:
            dateframe = "Jueves"
        if dateframe == 4:
            dateframe = "Viernes"
        if dateframe == 5:
            dateframe = "Sabado"
        if dateframe == 6:
            dateframe = "Domingo"
        dia_semana.append(dateframe)
        lemma = lemma.replace("[", '')
        lemma = lemma.replace("]", '')
        lemma = lemma.replace(" ", '')
        lemma_tag.append(lemma)
        clean = clean.replace("[", '')
        clean = clean.replace("]", '')
        clean = clean.replace(" ", '')
        clean_tag.append(clean)
        palabras = tweet.split(",")
        if len(palabras) < 3:
            short.append("Si")
        else:
            short.append("No")
    tweet_df["dia"] = dia_semana
    tweet_df["lemma_tag"] = lemma_tag
    tweet_df["clean_tag"] = clean_tag
    tweet_df["short"] = short
    tweet_df.to_csv(nombre_archivo, index=True, header=True)
        
sector = ['01','02','03','04','05','06','07','08','09','11','12']
for i in range(len(sector)):
    archivo_base = 'Tweets_' + sector[i] + '.csv'
    FormatearDatos(archivo_base, sector[i])

In [4]:
# Buscar y eliminar datos duplicados
sector = ['01','02','03','04','05','06','07','08','09','11','12']
for i in range(len(sector)-1,1,-1):
    nombre_archivo = 'Tweets_' + sector[i] + '.csv'
    tweet_df = pd.read_csv(nombre_archivo, index_col=0)
    if i != 6:
        aux_name = 'Tweets_' + sector[i-1] + '.csv'
        aux_df = pd.read_csv(aux_name, index_col=0)
        # eliminar copias existentes
        duplicados = pd.merge(tweet_df, aux_df, how='inner',on='created_at',left_index=True)
        tweet_df = tweet_df.drop(duplicados.index)
    elif i == 7:
        continue
    else:
        aux_name2 = 'Tweets_' + sector[i+1] + '.csv'
        aux_df2 = pd.read_csv(aux_name2, index_col=0)
        # eliminar copias existentes
        duplicados2 = pd.merge(tweet_df, aux_df2, how='inner',on='created_at', left_index=True)
        tweet_df = tweet_df.drop(duplicados.index)
    tweet_df.to_csv(nombre_archivo, index=True, header=True)

In [5]:
# Reunir datos de todos los sectores en una unica version
sector = ['01','02','03','04','05','06','07','08','09','11','12']
archivo_base =  'Tweets_' + sector[0] + '.csv'
df = pd.read_csv(archivo_base, index_col=0)
df.to_csv('Tweets.csv' , index=True, header=True)
for i in range(1,len(sector)):
    archivo = 'Tweets_' + sector[i] + '.csv'
    tweet_df = pd.read_csv(archivo, index_col=0)
    tweet_df.to_csv('Tweets.csv', mode='a', index=True, header=False)

In [5]:
from nltk import FreqDist
def get_all_words(cleaned_tokens_list):
    for token in cleaned_tokens_list:
            yield token
            
# Obtener todos los Hashtags presentes en cada uno de los datos por separado
archivo =  'Tweets.csv'
tuplas = []
tuplas.clear()
lista_hashtags = []
lista_hashtags.clear()
df = pd.read_csv(archivo, index_col=0)
for hashtags,sec,ltag,ctag,nlik,nrep,nret in zip(df['hashtags'],df['sector'],df['lemma_tag'],df['clean_tag'],df['nlikes'],df['nreplies'],df['nretweets']):
    hashtags = hashtags.replace('[','')
    hashtags = hashtags.replace(']','')
    hashtags = hashtags.replace(' ','')
    hashtag = hashtags.split(',')
    for tag in hashtag:
        lista_hashtags.append(tag)
        tuplas.append([tag,str(ltag),str(ctag),nlik,nrep,nret,sec])

all_hashtags = get_all_words(lista_hashtags)
freq_hashtags = FreqDist(all_hashtags)
list_hashtags = freq_hashtags.most_common(50)
print(list_hashtags)
        
'''df = pd.DataFrame(tuplas, columns = ['hashtags', 'lemma_tag', 'clean_tag', 'nlikes', 'nreplies', 'nretweets', 'sector'])  
df.to_csv('Hashtags.csv', index=True, header=False)'''

tuplas_hashtags = []
tuplas_hashtags.clear()
for i in range(50):
    tuplas_hashtags.append([list_hashtags[i]])
df = pd.DataFrame(tuplas_hashtags, columns = ['hashtag'])  
df.to_csv('HashtagsComunes.csv', index=True, header=True)

[('', 50635), ("'coronavirus'", 17456), ("'covid_19'", 15381), ("'quedateencasa'", 4084), ("'chile'", 3436), ("'covid19'", 2489), ("'cuidémonosentretodos'", 2396), ("'cuarentena'", 2321), ("'covid'", 1865), ("'covid__19'", 1722), ("'covidー19'", 1598), ("'coronaviruschile'", 1289), ("'covid19chile'", 1179), ("'coronavirusenchile'", 1064), ("'cuarentenatotal'", 996), ("'plancoronavirus'", 977), ("'pandemia'", 811), ("'covidー19chile'", 787), ("'covid2019'", 710), ("'iquique'", 681), ("'antofagasta'", 652), ("'yomequedoencasa'", 617), ("'covid2019chile'", 585), ("'quédateencasa'", 545), ("'cuidemonosentretodos'", 507), ("'quedateentucasa'", 463), ("'tarapacá'", 442), ("'ohiggins'", 439), ("'valdiviacl'", 435), ("'cuarentenatotalchile'", 406), ("'cuarentenacoronavirus'", 376), ("'cuarentenanacional'", 363), ("'viñadelmar'", 333), ("'toquedequeda'", 323), ("'altohospicio'", 321), ("'quedateenlacasa'", 320), ("'renunciapiñera'", 290), ("'yoapruebo'", 283), ("'laserena'", 279), ("'talca'", 274

In [15]:
archivo =  'Tweets.csv'
clean_class = []
clean_class.clear()
lemma_class = []
lemma_class.clear()
df = pd.read_csv(archivo, index_col=0)
for ltag,ctag in zip(df['lemma_tag'],df['clean_tag']):
    # Agregar clasificacion (etiqueta) adecuada mas exigente y filtrando datos neutros y no clasificables
    tag = ctag.split(',')
    if int(tag[0]) > (int(tag[1]) + 2):
        clean_class.append('Positivo')
    elif (int(tag[0]) + 2) < int(tag[1]):
        clean_class.append('Negativo')
    elif int(tag[0]) < 3 and int(tag[1]) < 3:
        clean_class.append('No Clasificable')
    else:
        clean_class.append('Neutro')
    tag = ltag.split(',')
    if int(tag[0]) > (int(tag[1]) + 2):
        lemma_class.append('Positivo')
    elif (int(tag[0]) + 2) < int(tag[1]):
        lemma_class.append('Negativo')
    elif int(tag[0]) < 3 and int(tag[1]) < 3:
        lemma_class.append('No Clasificable')
    else:
        lemma_class.append('Neutro')
df['clean_class'] = clean_class
df['lemma_class'] = lemma_class
df.drop(["date", "place", "language", "nlikes", "nreplies", "nretweets", "reply_to", "dia", "short"], inplace=True, axis=1)
df.to_csv('Tweets-2.csv', index=True, header=True)

In [3]:
archivo =  'Tweets.csv'
clean_class = []
clean_class.clear()
lemma_class = []
lemma_class.clear()
df = pd.read_csv(archivo, index_col=0)
for ltag,ctag in zip(df['lemma_tag'],df['clean_tag']):
    # Agregar clasificacion (etiqueta) adecuada mas exigente y filtrando datos neutros y no clasificables
    tag = ctag.split(',')
    if int(tag[0]) > (int(tag[1])):
        clean_class.append('Positivo')
    elif (int(tag[0])) < int(tag[1]):
        clean_class.append('Negativo')
    elif int(tag[0]) < 1 and int(tag[1]) < 1:
        clean_class.append('No Clasificable')
    else:
        clean_class.append('Neutro')
    tag = ltag.split(',')
    if int(tag[0]) > (int(tag[1])):
        lemma_class.append('Positivo')
    elif (int(tag[0])) < int(tag[1]):
        lemma_class.append('Negativo')
    elif int(tag[0]) < 1 and int(tag[1]) < 1:
        lemma_class.append('No Clasificable')
    else:
        lemma_class.append('Neutro')
df['clean_class'] = clean_class
df['lemma_class'] = lemma_class
# df.drop(["date", "place", "language", "nlikes", "nreplies", "nretweets", "reply_to", "dia", "short"], inplace=True, axis=1)
df.to_csv('Tweets.csv', index=True, header=True)